<a href="https://colab.research.google.com/github/laurenneal/capstone-visual-neuroscience/blob/Lauren/Validation_Wrapper_LN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install CaImAn - takes around 2 minutes

!git clone https://github.com/flatironinstitute/CaImAn.git
%cd '/content/CaImAn/'
!pip install -e .

# Install caiman dependencies (&> /dev/null will suppress the hundreds of printed lines in the output)
!pip install -r requirements.txt &> /dev/null

#import other dependencies
import cv2
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio

#IMPORTANT! Newer versions of h5py will cause errors when saving results
!pip install h5py==2.10.0
import h5py

#Set up caiman
!python setup.py build_ext -i

#Other file setup
!python caimanmanager.py install --inplace

#Caiman imports
import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
from caiman.summary_images import local_correlations_movie_offline
from scipy.ndimage import center_of_mass
from IPython.display import display, clear_output

Cloning into 'CaImAn'...
remote: Enumerating objects: 24933, done.
remote: Counting objects: 100% (868/868), done.
remote: Compressing objects: 100% (432/432), done.
remote: Total 24933 (delta 481), reused 753 (delta 419), pack-reused 24065
Receiving objects: 100% (24933/24933), 518.57 MiB | 28.31 MiB/s, done.
Resolving deltas: 100% (16694/16694), done.
Checking out files: 100% (317/317), done.
/content/CaImAn
Obtaining file:///content/CaImAn
  Running setup.py develop for caiman
     |████████████████████████████████| 2.9 MB 5.4 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
running build_ext
Installed /root/caiman_data


## Get paths to movie files and labelled ROI masks

In [3]:
#get a list of our masks and a list of our stacks in the same order
from os import listdir
maskpath = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs'
mask_filenames = [f for f in listdir(maskpath) if 'CLEAN' not in f and '.ipynb' not in f]
mask_filenames

['210731_0_1_manualROIs_mix2_syt.mat',
 '210815_0_1_manualROIs_mix1_syt.mat',
 '211106_1_1_manualROIs_tm4_t5_syt.mat',
 '211106_0_1_manualROIs_tm1_t5_syt.mat',
 '211023_0_1_manualROIs_tm2_tm4_syt.mat',
 '210728_0_1_manualROIs_tm2_tm9_syt.mat',
 'Carl Old Names']

In [4]:
stack_indices = [x[:10] for x in mask_filenames] #get the index portion of the masks
stack_celltypes = [x[22:(len(x)-4)] for x in mask_filenames] #get the cell type descriptions
stack_filenames = ['CLEAN_' + stack_indices[x] + '_stackRaw_mc_' + stack_celltypes[x] \
                   + '_.h5' for x in range(len(mask_filenames))] #reconstruct filenames for the movies we have masks for
stack_filenames

['CLEAN_210731_0_1_stackRaw_mc_mix2_syt_.h5',
 'CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5',
 'CLEAN_211106_1_1_stackRaw_mc_tm4_t5_syt_.h5',
 'CLEAN_211106_0_1_stackRaw_mc_tm1_t5_syt_.h5',
 'CLEAN_211023_0_1_stackRaw_mc_tm2_tm4_syt_.h5',
 'CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5',
 'CLEAN_Carl Old N_stackRaw_mc__.h5']

In [5]:
#checking that files exist in our cleaned file - only 2 do
stackpath = '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED'
stackfiles = [f for f in listdir(stackpath) if f in stack_filenames]
stackfiles

['CLEAN_210731_0_1_stackRaw_mc_mix2_syt_.h5',
 'CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5',
 'CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5',
 'CLEAN_211106_1_1_stackRaw_mc_tm4_t5_syt_.h5',
 'CLEAN_211106_0_1_stackRaw_mc_tm1_t5_syt_.h5',
 'CLEAN_211023_0_1_stackRaw_mc_tm2_tm4_syt_.h5']

In [6]:
#convert filenames back into paths
maskpaths = [maskpath+'/'+f for f in mask_filenames]
stackpaths = [stackpath+'/'+f for f in stackfiles]

In [7]:
#for now only keep the masks we cleaned have movies for - also in the correct order to match up
maskpaths = [maskpaths[x] for x in [1, 0, 4]]
maskpaths

['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210815_0_1_manualROIs_mix1_syt.mat',
 '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210731_0_1_manualROIs_mix2_syt.mat',
 '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/211023_0_1_manualROIs_tm2_tm4_syt.mat']

In [8]:
#join the lists into pairs of tuples
mask_stack_pairs = list(map(lambda x, y:[x,y], maskpaths, stackpaths))
mask_stack_pairs

[['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210815_0_1_manualROIs_mix1_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210731_0_1_stackRaw_mc_mix2_syt_.h5'],
 ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210731_0_1_manualROIs_mix2_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5'],
 ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/211023_0_1_manualROIs_tm2_tm4_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']]

## Initialize parameters object with starter values

In [9]:
#create parameters object
opts = params.CNMFParams()
#fname will be assigned in the loop
fnames = []
subfolder = 'stackRaw_mc'

In [10]:
# set initial values for extraction and evaluation
# most of these are specific to our data and will not need to be changed during optimization

# overall params about our data

fr = 20                 # approximate frame rate of data - CONFIRMED FPS
decay_time = .4         # length of transient - CONFIRMED APPROPRIATE FOR OUR INDICATOR GCaMP6f
dims = [128, 256]       # dimensions of the FOV in pixels - CONFIRMED
dxy = [.29, .29]        # resolution of 1 pixel in um - CONFIRMED BY CARL

opts.set('data', {'fnames': fnames,
                   'fr': fr,
                   'decay_time': decay_time,
                   'dims': dims,
                   'dxy': dxy
                  })


# params related to the temporal traces

p = 1                   # order of the autoregressive system - 0 from carl's code, probably should be 1
fudge_factor = 1        # (default is 0.96; Carl's value = 1) -- bias correction factor for discrete time constants
ITER = 5                # (default is 2; Carl's value=5) -- block coordinate descent iterations
tnb = 2                  # temporal global background components - TUNE

opts.set('temporal', {'p': p,
                      'fudge_factor': fudge_factor,
                      'ITER': ITER,
                      'nb': tnb
                 })

# p is also set in the preprocessing step
opts.set('preprocess', {'p': p
                 })



# params related to the FOV and patches for parallel processing

is_patches = True      # flag for processing in patches or not - turn on or off

if is_patches:          # PROCESS IN PATCHES AND THEN COMBINE 
    rf = 25             # half size of each patch -not tuned
    stride = 5          # overlap between patches -not tuned
    K = 3               # number of components in each patch - TUNE - gets set later in INIT params
    p_patch = p

else:                   # PROCESS THE WHOLE FOV AT ONCE
    rf = None           # setting these parameters to None Not used
    stride = None       # will run CNMF on the whole FOV not used
    K = 20              # number of neurons expected (in the whole FOV) - not used

n_processes = 2         # Number of processes to run in parallel, 2 for 2 cores available in Colab

opts.set('patch', {'rf': rf,
                   'stride': stride,
                   'n_processes': n_processes,
                   'p_patch': p_patch
                  })   



# initialization params
ssub = 2                # spatial downsampling
tsub = 2                # temporal downsampling
ssub_B = 3              # background spatial downsampling
gSig = [4,4]            # radius (half-size) of average neurons (in pixels)
nb = 2                  # number of background components

opts.set('init', {'K': K,            # declared above in patch params    
                   'gSig': gSig,      
                   'tsub': tsub,
                   'ssub': ssub,
                   'ssub_B': ssub_B,
                   'nb': nb
                  })

# parameters related to merging correlated ROIs
merge_thr = 0.9     # merging threshold, max correlation allowed - TUNE

opts.set('merging', {'merge_thr': merge_thr
                            })


#set some spatial params
snb = 2                  # spatial global background components - TUNE

opts.set('spatial', {'nb': snb
                            })

# %% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier (this will pick up only neurons
#           and filter out active processes)


#Not sure if these should be tuned or not

min_SNR = 2.5      # peak SNR for accepted components (if above this, acept)
SNR_lowest = 1         # minimum SNR for accepted components (if below this, reject)
rval_thr = 0.9     # space correlation threshold (if above this, accept)

use_cnn = False      # use the CNN classifier affects if 2 below params are used
min_cnn_thr = 0.9  # if cnn classifier predicts below this value, reject
cnn_lowest = 0.1   # neurons with cnn probability lower than this value are rejected

opts.set('quality', {'min_SNR': min_SNR,
                     'SNR_lowest': SNR_lowest,
                     'rval_thr': rval_thr,
                     'use_cnn': use_cnn,
                     'min_cnn_thr': min_cnn_thr,
                     'cnn_lowest': cnn_lowest})


#Manually assign subfolder variable
opts.motion['var_name_hdf5'] = subfolder
opts.data['var_name_hdf5'] = subfolder

## Define pipeline function to run Caiman

In [11]:
#Function to run cnmf, run seeded cnmf using masks, then return validation

def score_params(path_to_stack, path_to_masks, opts):
  import warnings
  warnings.simplefilter(action='ignore', category=FutureWarning)

  fnames = [path_to_stack]

  try:

    if 'dview' in locals():
      cm.stop_server(dview=dview)
    dview = cm.cluster.start_server(ncpus=2)

    #Run CNMF on raw stack using passed params
    #print('starting cnmf')
    cnm = cnmf.CNMF(n_processes = 2, params=opts, dview=dview)
    #print('cnm object initialized')
    cnm = cnm.fit_file(motion_correct = False, include_eval=True)
    #print('cnmf and component evaluation completed')


    #Read in masks and reformat
    g = h5py.File(path_to_masks, 'r')

    #transpose the matrix and save to an array A
    mask_A = g['bwMaskStack'][:].T

    g.close()

    #rearrange the dimensions and show the new shape
    mask_A = mask_A.transpose(1,0,2)

    #reshape to 2D, first dimension is 128*256 (32768), 2nd dimension is the # of ROI's
    mask_A = mask_A.reshape((mask_A.shape[1]*mask_A.shape[0]), mask_A.shape[2])

    #convert the values from 0/1 to boolean False/True
    mask_A = np.array(mask_A, dtype=bool)
    #print('mask read in and reformatted')

    #Seeded CNMF only works when seeded using mmap
    #import pathlib

    #read data from the original cnmf's mmap location
    Yr, dims, T = cm.load_memmap(cnm.mmap_file)

    # load frames in python format (T x X x Y)
    mov = np.reshape(Yr.T, [T] + list(dims), order='F')
    #print('movie matrix loaded for seeded cnmf')


    #For seeded CNMF, need to adjust some params
    opts.patch['only_init'] = False
    opts.data['use_cnn'] = False

    #print('params adjusted for seeded cnmf')
    
    #Initialize a new cnmf object and pass in our masks as the "Ain" param
    #"Ain" is A-in, meaning the A matrix holding the spatial footprints of the roi's
    cnm_seeded = cnmf.CNMF(n_processes = 2, params=opts, dview=None, Ain=mask_A)
    #print('seeded cnmf object initialized')
    cnm_seeded.fit(mov)
    #print('seeded cnmf completed')

    #Try running register_ROIs and see what happens
    matched_ROIs1, matched_ROIs2, non_matched1, non_matched2, performance, A2 = cm.base.rois.register_ROIs(cnm_seeded.estimates.A, cnm.estimates.A, dims=dims)
    #print('validation and scoring completed')

    #restore params for the next regular cnmf iteration
    opts.patch['only_init'] = True
    #opts.data['use_cnn'] = True

    cm.stop_server(dview=dview)


    #return performance metrics
    return {'matched_ROIs1': matched_ROIs1,
            'matched_ROIs2': matched_ROIs2,
            'non_matched1': non_matched1,
            'non_matched2': non_matched2, 
            'performance': performance,
            'A2': A2,
            'cnm_estimates': cnm.estimates,
            'seeded_cnm_estimates': cnm_seeded.estimates}

  except:
    print('failed')
    cm.stop_server(dview=dview)

## Function to score all our stacks and return the avg f1

In [12]:
#loop through the stack-mask pairs and print the accuracy results

def score_all_stacks(all_params):

  print(all_params)

  K = all_params[0]
  nb = all_params[1]
  merge_thresh = all_params[2]
  gSig = [all_params[3], all_params[4]] #param passes one value, recreate [x,y] structure
  SNR_lowest = all_params[5]


  from statistics import mean

  #hard coded paths since these won't change for now, proceduralize later - 1 just to test
  mask_stack_pairs = [['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/211106_1_1_manualROIs_tm4_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5'],
 ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210815_0_1_manualROIs_mix1_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5'],
 ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat',
  '../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']]



  #Assign params passed in by the wrapper function
  opts.init['K'] = K
  opts.init['nb'] = nb #nb gets assigned in three different dicts
  opts.spatial['nb'] = nb
  opts.temporal['nb'] = nb
  opts.merging['merge_thresh'] = merge_thresh
  opts.init['gSig'] = gSig

  #store precisions from each stack-mask pair for this iteration in a list, we'll return the average
  f1s = []

  #this loop will run all stacks through caiman using a given set of parameters to get the accuracy
  for pair in mask_stack_pairs:
    path_to_stack = pair[1]
    path_to_masks = pair[0]

    #set fnames before calling caiman function
    opts.data['fnames'] = [path_to_stack]
    print(opts.data['fnames'])

    results = score_params(path_to_stack=path_to_stack, \
              path_to_masks=path_to_masks, \
              opts=opts)
    
    #store this stack-mask pair's precision
    #later handle some kind of logging to keep a dict of info returned
    f1s.append(results['performance']['f1_score'])

    print(results['performance'])

  #return the mean precision for all mask-stack pairs
  #negative of the precision, so that the optimizer can minimize
  print(f1s)
  return -mean(f1s)



## Bayesian Model Optimization

In [13]:
!pip install scikit-optimize
import skopt
from skopt.callbacks import CheckpointSaver
from skopt.callbacks import DeltaYStopper
from skopt.callbacks import DeadlineStopper
from skopt import plots
from skopt import load

     |████████████████████████████████| 100 kB 2.4 MB/s 


In [ ]:
# #load old checkpoints if we need - change path
# checkpoints = skopt.load('../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/search_checkpoints2.pkl')

In [14]:
#define the parameter space - this is the entirety of the space I want to explore and will take a long time to run

#change params if needed
SPACE = [
        skopt.space.Integer(10, 20, name='K'), #number of ROIs to expect in the FOV
        skopt.space.Integer(1,3, name='nb'), #measure of how much background noise to remove, minimum 2 but higher could remove ROIs
        skopt.space.Real(.90, .95, name= 'merge_thresh'), #how correlated ROIs need to be before they get merged
        skopt.space.Integer(1, 3, name = 'gSig_wid'), #half-width of neurons - Supposedly the most important
        skopt.space.Integer(1, 3, name = 'gSig_hei'), #half-height of neurons - Supposedly the most important
        skopt.space.Real(.5, 1.5, name = 'SNR_lowest')] #Minimum SNR accepted for ROIs, (if below this, reject)

In [15]:
#saves a checkpoint every iteration to this filepath

#CHANGE THIS PATH OR FILENAME BEFORE YOU RUN THIS - LEAVE THE EXTENSION AS .PKL
checkpoint_saver = CheckpointSaver('../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/search_checkpoints_ln_attempt_2.pkl')

#stopping criteria - stop if the 5 best iterations are within 3% of each other- might want to make more strict
stopping_criteria = skopt.callbacks.DeltaYStopper(.03, n_best=5)

#second criteria, stop after 11.5 hours if it hasn't stopped yet - takes time in seconds as argument
mins = 11.5 * 60
seconds = mins * 60
stopping_criteria_2 = DeadlineStopper(total_time= seconds)


# surrogate function, directly pass in parameter list, returns precision result directly
#this is the function that the optimizer minimizes
#score_all_stacks returns the precision * -1 so that the optimizer can minimize it
def objective(parameters):
    return score_all_stacks(parameters)

In [16]:
#optimizer using objective function, parameter space, checkpointer, stopping criteria

#change minimizer function if needed
results = skopt.gp_minimize(objective, 
                              SPACE, 
                              callback=[checkpoint_saver, stopping_criteria, stopping_criteria_2], 
                              initial_point_generator = 'halton',
                              n_jobs = 2,
                              verbose=True, )

Iteration No: 1 started. Evaluating function at random point.
[10, 1, 0.9, 1, 1, 0.5]
['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5']
Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
........

../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5


failed
....

TypeError: ignored

In [ ]:
results

In [ ]:
skopt.plots.plot_convergence(results)

In [ ]:
skopt.plots.plot_objective(results)

## Old working code

In [ ]:
# #tests score_all_stacks directly
# dummy_in = [13, 2, 0.9, 5, 10, .8]
# score_all_stacks(dummy_in)

[13, 2, 0.9, 5, 10, 0.8]
['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_211106_1_1_stackRaw_mc_tm4_syt_.h5']
Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
............spatial support for each components given by the user
{'recall': 0.2, 'precision': 0.07692307692307693, 'accuracy': 0.058823529411764705, 'f1_score': 0.1111111111111111}
['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210815_0_1_stackRaw_mc_mix1_syt_.h5']
Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
............spatial support for each components given by the user
{'recall': 0.13333333333333333, 'precision': 0.16666666666666666, 'accuracy': 0.08, 'f1_score': 0.14814814814814814}
['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']
Waiting for connection file: ~/.ipython/profile_default/security/ipc

-0.1397530864197531

In [ ]:
# #tests score_params directly
# opts.data['fnames'] = ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']
# score_params(path_to_masks='../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat', \
#               path_to_stack='../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5', \
#               opts=opts)

USING MODEL:/root/caiman_data/model/cnn_model.json
1/1 [==============================] - 0s 127ms/step
....spatial support for each components given by the user


{'A2': <32768x20 sparse matrix of type '<class 'numpy.float64'>'
 	with 11222 stored elements in Compressed Sparse Column format>,
 'cnm_estimates': <caiman.source_extraction.cnmf.estimates.Estimates at 0x7f5d45c68210>,
 'matched_ROIs1': array([10]),
 'matched_ROIs2': array([3]),
 'non_matched1': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11]),
 'non_matched2': array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19]),
 'performance': {'accuracy': 0.03225806451612903,
  'f1_score': 0.0625,
  'precision': 0.05,
  'recall': 0.08333333333333333},
 'seeded_cnm_estimates': <caiman.source_extraction.cnmf.estimates.Estimates at 0x7f5d45b83090>}

In [ ]:
# #tests cnm directly
# opts.quality['use_cnn'] = True
# opts.data['fnames'] = ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']

# if 'dview' in locals():
#     cm.stop_server(dview=dview)
# dview = cm.cluster.start_server(ncpus=2)
# cnm = cnmf.CNMF(n_processes=2, params=opts, dview=dview)
# #print('cnm object initialized')
# cnm = cnm.fit_file(motion_correct = False, include_eval=True)
# cm.stop_server(dview=dview)

# cnm.estimates.A

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
........USING MODEL:/root/caiman_data/model/cnn_model.json
1/1 [==============================] - 0s 134ms/step
....

<32768x20 sparse matrix of type '<class 'numpy.float64'>'
	with 8564 stored elements in Compressed Sparse Column format>

In [ ]:
# #tests masks and scoring directly 
# fnames = ['../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/stackRaw/CLEANED/CLEAN_210728_0_1_stackRaw_mc_tm2_tm9_syt_.h5']
# path_to_masks='../drive/MyDrive/DS6011_Capstone_VisualNeuroscience/DATA/manualROIs/210728_0_1_manualROIs_tm2_tm9_syt.mat'
# #Read in masks and reformat
# g = h5py.File(path_to_masks, 'r')

# #transpose the matrix and save to an array A
# mask_A = g['bwMaskStack'][:].T

# g.close()

# #rearrange the dimensions and show the new shape
# mask_A = mask_A.transpose(1,0,2)

# #reshape to 2D, first dimension is 128*256 (32768), 2nd dimension is the # of ROI's
# mask_A = mask_A.reshape((mask_A.shape[1]*mask_A.shape[0]), mask_A.shape[2])

# #convert the values from 0/1 to boolean False/True
# mask_A = np.array(mask_A, dtype=bool)
# #print('mask read in and reformatted')

# if 'dview' in locals():
#     cm.stop_server(dview=dview)
# dview = cm.cluster.start_server(ncpus=2)

# #Seeded CNMF only works when seeded using mmap
# #import pathlib

# #create memmory map location for the original movie
# #fname_new = cm.save_memmap(fnames, base_name=pathlib.Path(fnames[0]).stem + "_memmap_", order='C')

# #read data from mmap location
# Yr, dims, T = cm.load_memmap(cnm.mmap_file)

# # load frames in python format (T x X x Y)
# mov = np.reshape(Yr.T, [T] + list(dims), order='F')
# #print('movie matrix loaded for seeded cnmf')


# #For seeded CNMF, need to adjust some params
# opts.patch['only_init'] = False
# opts.patch['rf'] = None
# opts.data['use_cnn'] = False

# #print('params adjusted for seeded cnmf')
    
# #Initialize a new cnmf object and pass in our masks as the "Ain" param
# #"Ain" is A-in, meaning the A matrix holding the spatial footprints of the roi's
# cnm_seeded = cnmf.CNMF(n_processes = 2, params=opts, dview=None, Ain=mask_A)
# #print('seeded cnmf object initialized')
# cnm_seeded.fit(mov)
# #print('seeded cnmf completed')

# #Try running register_ROIs and see what happens
# matched_ROIs1, matched_ROIs2, non_matched1, non_matched2, performance, A2 = cm.base.rois.register_ROIs(cnm_seeded.estimates.A, cnm.estimates.A, dims=dims)
# #print('validation and scoring completed')

# performance

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
........spatial support for each components given by the user


{'accuracy': 0.06666666666666667,
 'f1_score': 0.125,
 'precision': 0.1,
 'recall': 0.16666666666666666}